### Test if filter_lifestyle_experts works

In [2]:
%pwd
%cd ..

c:\Users\admin\Documents\__uni\Cal\4-1\CS 194-280\mental_health_therapy


In [16]:
# llm_utils.py
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

from google import genai

load_dotenv(find_dotenv())
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# print(GEMINI_API_KEY)
client = genai.Client(api_key=GEMINI_API_KEY)

def call_gemini(prompt, model="gemini-1.5-flash-8b"):
    response = client.models.generate_content(
        model=model, contents=prompt
    )
    return response.text

def filter_lifestyle_experts(chat_history, lifestyle_expert_list):
    prompt = f"""Does the following conversation mention or imply [{', '.join(lifestyle_expert_list)}]. Reply "Yes" or "No" for each topic in the following format (replacing name of agent with the element in the list):
    name of agent 1: Yes
    name of agent 2: No
    
    Conversation:
    {chat_history}"""
    response = call_gemini(prompt)
    result = response.split("\n")[:-1]
    return result
    

In [17]:
# test case
chat = "I am struggling with my studies and feel tired."
experts = ["work/study", "exercise", "relationships"]
result = filter_lifestyle_experts(chat, experts)
print(result)

['work/study: Yes', 'exercise: No', 'relationships: No']


### Categories that passes filter applies to experts

In [28]:
from crewai import Agent, Task, Crew
import os

therapy_agent = Agent(
    role="Conversational Therapy Agent",
    goal="Provide supportive, evidence-based therapeutic conversations",
    backstory="Trained on therapy dialogues, and skilled in CBT and motivational interviewing",
    llm="gemini/gemini-1.5-flash-8b",  # or your preferred model,
    verbose=True

)

workstudy_agent = Agent(
    role="Work, study, occupation expert",
    goal="Provide detailed feedback to the patient's struggles and analyze pain points, and discuss what the patient can do going forward. Focus on building good long term habits and provide actionable steps.",
    backstory="You are an expert psychologist on good work ethics. You are in charge of distilling this knowledge to the patient if he struggles with such struggles.",
    llm="gemini/gemini-1.5-flash-8b",  # or your preferred model
    verbose=True
)

exercise_agent = Agent(
    role="Physical Exercise Expert",
    goal="Provide advice and feedback only when the patient discusses exercise, physical activity, or related struggles.",
    backstory="You are a specialist in physical wellbeing. Only intervene if the conversation is relevant to your domain.",
    llm="gemini/gemini-1.5-flash-8b",
    verbose=True
)

relationship_agent = Agent(
    role="Relationship and Social Connection Expert",
    goal="Offer insights and support when the patient discusses relationships, social interactions, or interpersonal challenges. Help the patient build healthy communication skills, navigate conflicts, and strengthen supportive connections.",
    backstory="You are a psychologist specializing in relationships, social dynamics, and emotional intelligence. You provide guidance only when the patient brings up issues related to family, friends, romantic partners, or social situations.",
    llm="gemini/gemini-1.5-flash-8b",
    verbose=True
)


In [25]:
# Map agent names to agent objects
agent_map = {
    "work/study": workstudy_agent,
    "exercise": exercise_agent,
    "relationships": relationship_agent,
}

# Parse result and select agents
relevant_agents = []
relevant_tasks = []

for res in result:
    topic, verdict = res.split(":")
    topic = topic.strip()
    verdict = verdict.strip().lower()
    if verdict == "yes":
        agent = agent_map[topic]
        relevant_agents.append(agent)
        # Create a task for this agent
        task = Task(
            description=f"Analyze and provide support for the following user message: '{chat}'",
            expected_output="A concise, actionable response tailored to the user's concern.",
            agent=agent
        )
        relevant_tasks.append(task)


In [ ]:
# crew = Crew(
#     agents=relevant_agents,
#     tasks=relevant_tasks,
#     verbose=True
# )

# result = crew.kickoff()
# print(result)


In [30]:
# --- Create the final synthesis task ---
# This task explicitly uses the outputs of the expert tasks as context.
synthesis_task = Task(
    description=f"Review the analyses provided by the expert agents regarding the user's message: '{chat}'. \
                Synthesize these distinct perspectives into a single, cohesive, and supportive therapeutic response. \
                **Do not just list or copy the expert outputs.** Integrate their key points naturally into your own compassionate and actionable feedback.",
    expected_output="A final, integrated therapeutic response that acknowledges the user's struggles (studies, tiredness, lack of exercise) and incorporates relevant synthesized insights from the specialists.",
    agent=therapy_agent,
    context=relevant_tasks  # <-- CRITICAL: This tells CrewAI to feed the outputs of expert_tasks here
)


In [ ]:
from crewai import Process

# Include the therapy agent PLUS all relevant expert agents
all_involved_agents = [therapy_agent] + relevant_agents

# The tasks list includes all expert tasks FIRST, then the synthesis task LAST
all_tasks = relevant_tasks + [synthesis_task]

# Create the crew
crew = Crew(
    agents=all_involved_agents,
    tasks=all_tasks,
    process=Process.sequential, # Ensures experts run before the synthesis task due to context dependency
    verbose=True
    memory=True, # Enable memory
)

# Kickoff the crew
print("\n--- Kicking off the Crew ---")
final_result = crew.kickoff(inputs={'chat': chat}) # Pass inputs if needed by descriptions

print("\n--- Final Synthesized Response ---")
print(final_result)



--- Kicking off the Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d0bc3c02-67b2-4fe2-9635-1a880b46e727                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 50613427-f8d9-4ecb-a59d-cbd2786b0368
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 50613427-f8d9-4ecb-a59d-cbd2786b0368
       Status: Executing Task...
    └── 🤖 Agent: Work, study, occupation expert
            Status: In Progress

# Agent: Work, study, occupation expert
## Task: Analyze and provide support for the following user message: 'I am struggling with my studies and feel tired.'


# Agent: Work, study, occupation expert
## Final Answer: 
It's completely understandable to feel tired and struggle with studies.  Feeling overwhelmed is a common experience, and it's crucial to address the underlying factors contributing to this.  Let's break down some potential issues and actionable steps.

**Potential Pain Points:**

* **Lack of Structure/Poor Time Management:**  Are you struggling to allocate time effectively for studying, or do tasks feel overwhelming?
* **Poor Study Habits:**  Are you using effective study techniques (e.g., active recall, spaced repetition)?  Do you get easily distracted?
* **Underlying Medical Conditions:**  Sometimes fatigue can stem from conditions like anemia, sleep disorders, or other health issues.  It's worth considering this.
* **Burnout/Stress:**  Is your workload manageable, or 

...

🚀 Crew: crew
├── 📋 Task: 50613427-f8d9-4ecb-a59d-cbd2786b0368
│      Assigned to: Work, study, occupation expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Work, study, occupation expert
│           Status: ✅ Completed
└── 📋 Task: 8f22663a-eb3e-41e2-beb3-df67448d3b10
       Status: Executing Task...
    └── 🤖 Agent: Conversational Therapy Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 50613427-f8d9-4ecb-a59d-cbd2786b0368
│      Assigned to: Work, study, occupation expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Work, study, occupation expert
│           Status: ✅ Completed
└── 📋 Task: 8f22663a-eb3e-41e2-beb3-df67448d3b10
       Assigned to: Conversational Therapy Agent
       Status: ✅ Completed
    └── 🤖 Agent: Conversational Therapy Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8f22663a-eb3e-41e2-beb3-df67448d3b10                                                                     │
│  Agent: Conversational Therapy Agent                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d0bc3c02-67b2-4fe2-9635-1a880b46e727                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Synthesized Response ---
"It's completely understandable that you're feeling tired and struggling with your studies.  Feeling overwhelmed is a very common experience, and it's important to address the contributing factors.  Let's work together to break down these challenges into manageable steps.

First, let's explore how you're spending your time.  It sounds like you might be feeling overwhelmed with the workload, so  tracking how you spend your time for a week can help you see where you might be losing efficiency.  What's your typical daily schedule, and what activities are stealing your focus or energy?  Are you setting realistic goals? Identifying the specific distractions, like social media or a noisy environment, is also key.  Journaling about your feelings when studying can help you understand what triggers feelings of tiredness or disinterest.  By understanding your patterns, you can start to implement adjustments.

Next, let's examine your study habits.  Using effec

### Telegram integration